In [9]:
from nuscenes.nuscenes import NuScenes  
import numpy as np
import os
import sys

In [10]:
# NuScenes 데이터셋 경로 설정
nusc_path = "/home/noh/pgc/datasets/nuscenes/v1.0-mini"

# NuScenes 인스턴스 생성
nusc = NuScenes(version='v1.0-mini', dataroot=nusc_path, verbose=True)

# NuScenes scenes 목록 출력
nusc.list_scenes()

Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 1.140 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
scene-0061, Parked truck, construction, intersectio... [18-07-24 03:28:47]   19s, singapore-onenorth, #anns:4622
scene-0103, Many peds right, wait for turning car, ... [18-08-01 19:26:43]   19s, boston-seaport, #anns:2046
scene-0655, Parking lot, parked cars, jaywalker, be... [18-08-27 15:51:32]   20s, boston-seaport, #anns:2332
scene-0553, Wait at intersection, bicycle, large tr... [18-08-28 20:48:16]   20s, boston-seaport, #anns:1950
scene-0757, Arrive at busy intersection, bus, wait ... [18-08-30 19:25:08]   20s, boston-seaport, #anns:592
scene-0796, Scooter, peds on sidewalk, bus, cars, t... [18-10-02 02:52:24]   20s, sing

In [11]:
for scene in nusc.scene:
    token = scene['first_sample_token']
    frame_count = 0
    while token:
        frame_count += 1
        token = nusc.get('sample', token)['next']
    print(f"{scene['name']}: {frame_count} frames")

scene-0061: 39 frames
scene-0103: 40 frames
scene-0553: 41 frames
scene-0655: 41 frames
scene-0757: 41 frames
scene-0796: 40 frames
scene-0916: 41 frames
scene-1077: 41 frames
scene-1094: 40 frames
scene-1100: 40 frames


In [20]:
def load_scenes(nusc):
    sample_dict = dict()
    sample_list = list()
    
    for scene in nusc.scene:
        scene_name = scene['name']
        token = scene['first_sample_token']
        print(f"[Processing {scene_name}...]")

        sample_list = list()
        frame_count = 0
        
        while token:
            frame = f"{frame_count:02d}"
            sample = nusc.get('sample', token)
            lidar_data = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
            lidar_path = nusc.get_sample_data_path(lidar_data['token'])

            bin_ply_path = f"{nusc_path}/ply/bin/{scene_name}/{scene_name}_{frame}.ply"
            ascii_ply_path = f"{nusc_path}/ply/ascii/{scene_name}/{scene_name}_{frame}_ascii.ply"

            sample_list.append(
                {
                    'scene':    scene_name,
                    'frame':    frame,
                    'token':    token,
                    'lidar_path': lidar_path,
                    'bin_ply_path': bin_ply_path,
                    'ascii_ply_path': ascii_ply_path
                }
            )

            token = sample['next']
            frame_count += 1

        print(f">>> {frame_count} frames processed.")
        sample_dict[scene_name] = sample_list         
    return sample_dict            

        

In [21]:
import json

sample_json = load_scenes(nusc)
json_path = f"{nusc_path}/nuscenes.json"

with open(json_path, "w") as f:
    json.dump(sample_json, f, indent=4, ensure_ascii=False)



[Processing scene-0061...]
>>> 39 frames processed.
[Processing scene-0103...]
>>> 40 frames processed.
[Processing scene-0553...]
>>> 41 frames processed.
[Processing scene-0655...]
>>> 41 frames processed.
[Processing scene-0757...]
>>> 41 frames processed.
[Processing scene-0796...]
>>> 40 frames processed.
[Processing scene-0916...]
>>> 41 frames processed.
[Processing scene-1077...]
>>> 41 frames processed.
[Processing scene-1094...]
>>> 40 frames processed.
[Processing scene-1100...]
>>> 40 frames processed.


In [22]:
from plyfile import PlyData, PlyElement

def extract_ply(json, mode=0):
    source = json['lidar_path']
    
    if mode == 0: # binary
        ply_file = json['bin_ply_path']
    else:
        ply_file = json['ascii_ply_path']
    
    ply_dir = os.path.dirname(ply_file)
    if not os.path.exists(ply_dir):
        os.makedirs(ply_dir, exist_ok=True)

    try:
        data = np.fromfile(source, dtype=np.float32).reshape([-1,5])

        point = data[:, :3]
        refc = data[:, 3]
        ring = data[:, 4]

        vertex = np.empty(point.shape[0], dtype=[
            ('x', 'f4'),
            ('y', 'f4'),
            ('z', 'f4'),
            ('refc', 'u1'),
            ('ring', 'u1')
        ])

        vertex['x'] = point[:, 0]
        vertex['y'] = point[:, 1]
        vertex['z'] = point[:, 2]
        vertex['refc'] = refc
        vertex['ring'] = ring

        plydata = PlyData([PlyElement.describe(vertex, 'vertex')], text=mode)
        plydata.write(ply_file)
    
    except FileNotFoundError:
        print(f"File not found: {source}")
        sys.exit(1)

In [23]:
nuscenes_json = json_path

if not os.path.exists(nuscenes_json):
    raise ValueError("nuscenes.json does not exist")

else:
    with open(nuscenes_json, 'r') as f:
        data = json.load(f)
        for scene_name, frames in data.items():
            for frame in frames:
                extract_ply(frame, 0)
            print(f"Extracted the {scene} point clouds.")

Extracted the {'token': 'e233467e827140efa4b42d2b4c435855', 'log_token': '8fefc430cbfa4c2191978c0df302eb98', 'nbr_samples': 40, 'first_sample_token': 'a480496a5988410fbe3d8ed6c84da996', 'last_sample_token': 'abf3d91d3c28407e80e3334fe89c03cb', 'name': 'scene-1100', 'description': 'Night, peds in sidewalk, peds cross crosswalk, scooter, PMD, difficult lighting'} point clouds.
Extracted the {'token': 'e233467e827140efa4b42d2b4c435855', 'log_token': '8fefc430cbfa4c2191978c0df302eb98', 'nbr_samples': 40, 'first_sample_token': 'a480496a5988410fbe3d8ed6c84da996', 'last_sample_token': 'abf3d91d3c28407e80e3334fe89c03cb', 'name': 'scene-1100', 'description': 'Night, peds in sidewalk, peds cross crosswalk, scooter, PMD, difficult lighting'} point clouds.
Extracted the {'token': 'e233467e827140efa4b42d2b4c435855', 'log_token': '8fefc430cbfa4c2191978c0df302eb98', 'nbr_samples': 40, 'first_sample_token': 'a480496a5988410fbe3d8ed6c84da996', 'last_sample_token': 'abf3d91d3c28407e80e3334fe89c03cb', 'na